In [548]:
import time
import random
#from PyOCR import *
#from Summarizer import *
import subprocess
from os import listdir
from os.path import isfile, join
import os.path
import sklearn.datasets
import nltk.stem
import numpy as np
import pandas as pd
from stop_words import get_stop_words
import re
import scipy as sp
import PyPDF2
import docx
import sys
import os
import os.path
import string
import docx
import docx2txt
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

In [549]:
stop_words = get_stop_words('english')
english_stemmer = nltk.stem.SnowballStemmer('english')

In [550]:
def file_operation():
    data = []
    mypath = "Users/sahebsingh/Desktop/Projects/Honeywell/Honeywell" #Path to folder/Directory
    
    list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for k in range(len(list_files)):
        document = docx.Document(list_files[k])
    print (document.paragraphs)

In [551]:
def getPdf(file):
    data = []
    file = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(file)
    page_number = pdfReader.numPages
    for i in range(0, page_number):
        pageObj = pdfReader.getPage(i)
        data.append(pageObj.extractText())
        
    return '\n'.join(data)
    

In [552]:
def getDocx(filename):
    my_text = docx2txt.process(filename)
    
    return np.array(my_text)
    

In [553]:
data1 = getDocx('2.docx')

In [554]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
    
    

In [568]:
data4 = getPdf('7.pdf')

In [569]:
import nltk
# nltk.download('punkt')
# nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest


def summarize(text, n):
    
    sents = sent_tokenize(text)
    #assert n <= len(sents)
    if(n>=len(sents)):
        n = random.randint(int(len(sents)/2), len(sents))
    
    wordSent = word_tokenize(text.lower())
    stopWords = set(stopwords.words('english')+list(punctuation))
    
    wordSent= [word for word in wordSent if word not in stopWords]
    freq = FreqDist(wordSent)
    ranking = defaultdict(int)
    
    for i, sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
    sentsIDX = nlargest(n, ranking, key=ranking.get)
    print('Summarise Completed')
    return [sents[j] for j in sorted(sentsIDX)]

summaryArr = summarize(data4, 20)
# summaryArr

Summarise Completed


In [571]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
def Vect(data, T):
    
    vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')
    X = vectorizer.fit_transform(summaryArr)
    km = KMeans(n_clusters = T, init = 'k-means++', max_iter = 100, n_init = 2, verbose = True)
    km.fit(X)
    np.unique(km.labels_, return_counts=True)
    text={}
    for i,cluster in enumerate(km.labels_):
        oneDocument = summaryArr[i]
        if cluster not in text.keys():
            text[cluster] = oneDocument
        else:
            text[cluster] += oneDocument
    stopWords = set(stopwords.words('english')+list(punctuation))
    keywords = {}
    counts={}
    for cluster in range(T):
        word_sent = word_tokenize(text[cluster].lower())
        word_sent=[word for word in word_sent if word not in stopWords]
        freq = FreqDist(word_sent)
        keywords[cluster] = nlargest(100, freq, key=freq.get)
        counts[cluster]=freq
        uniqueKeys={}
    for cluster in range(T):   
        other_clusters=list(set(range(T))-set([cluster]))
        keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
        unique=set(keywords[cluster])-keys_other_clusters
        uniqueKeys[cluster]=nlargest(10, unique, key=counts[cluster].get)
    #print(uniqueKeys)
    print(" ")
    print(" ")
    print('Vect Completed')
    return [X, km, vectorizer]
Vect(data, 10)

Initialization complete
Iteration  0, inertia 7.344
Iteration  1, inertia 3.944
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 7.216
Iteration  1, inertia 4.013
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed


[<20x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 480 stored elements in Compressed Sparse Row format>,
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
     n_clusters=10, n_init=2, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=True),
 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=2,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None)]

In [573]:
Z = Vect(data, 10)
X = Z[0]
km = Z[1]
vectorizer = Z[2]

Initialization complete
Iteration  0, inertia 6.843
Iteration  1, inertia 3.534
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
Initialization complete
Iteration  0, inertia 7.031
Iteration  1, inertia 3.882
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.719806e-07
 
 
Vect Completed


In [574]:
post = """ his document is to provide an example of a weight and balance document for an aircraft type certificate application in accordance with CS-LSA. The document can be used even if the applicant does not own a DOA. It does not substitute, in any of its parts, the prescriptions of Part-21 and its amendments.
This document is intended to assist applicants in applying for an LSA RTC/TC and therefore demonstrating compliance of the design to the requirements.
The document should not be read as a template and it should not be used as a form to fill. The content shall be checked for appropriateness and changed accordingly by the applicant.
The required information can be presented entirely in this document, or in additional documents appropriately identified and referred to.
"""

In [575]:
new_post_vec = vectorizer.transform([post])
print(new_post_vec)
#new_post_label = km.predict(new_post_vec)[0]#We will predict it's cluster 

#similar_indices = (km.labels_ == new_post_label).nonzero()[0]


  (0, 118)	0.177560834545
  (0, 115)	0.214809079162
  (0, 112)	0.164266360598
  (0, 88)	0.328532721196
  (0, 58)	0.134700765553
  (0, 50)	0.134700765553
  (0, 49)	0.762108695881
  (0, 46)	0.143289237078
  (0, 44)	0.164266360598
  (0, 40)	0.177560834545
  (0, 29)	0.134700765553
  (0, 27)	0.214809079162
  (0, 23)	0.164266360598


In [576]:
# Old Method, Not Doing that anymore
similar = []
for i in similar_indices:
    print(X[i])
    dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
    similar.append((dist, summaryArr[i]))


  (0, 85)	0.140989189967
  (0, 18)	0.140989189967
  (0, 115)	0.140989189967
  (0, 69)	0.315225676838
  (0, 131)	0.140989189967
  (0, 79)	0.140989189967
  (0, 23)	0.107815652903
  (0, 134)	0.127220928102
  (0, 32)	0.233082869028
  (0, 133)	0.127220928102
  (0, 10)	0.140989189967
  (0, 80)	0.107815652903
  (0, 132)	0.140989189967
  (0, 90)	0.236419257628
  (0, 136)	0.140989189967
  (0, 11)	0.127220928102
  (0, 94)	0.157612838419
  (0, 66)	0.215631305807
  (0, 137)	0.116541434514
  (0, 13)	0.140989189967
  (0, 138)	0.116541434514
  (0, 14)	0.140989189967
  (0, 141)	0.100438109672
  (0, 135)	0.088410377705
  (0, 140)	0.116541434514
  (0, 16)	0.281978379933
  (0, 139)	0.116541434514
  (0, 34)	0.127220928102
  (0, 63)	0.107815652903
  (0, 84)	0.127220928102
  (0, 71)	0.140989189967
  (0, 53)	0.215631305807
  (0, 59)	0.116541434514
  (0, 35)	0.116541434514
  (0, 40)	0.116541434514
  (0, 120)	0.140989189967
  (0, 1)	0.116541434514
  (0, 51)	0.088410377705
  (0, 58)	0.088410377705
  (0, 50)	0.0

In [577]:
# New Method
similar = []
for i in range(0,20):
    dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
    similar.append((dist, summaryArr[i]))

In [578]:
similar = sorted(similar) 
print(len(similar))
show_at_1 = similar[0]
show_at_2 = similar[1]
show_at_3 = similar[2]

20


In [579]:
print("=== #1 ===")
print(show_at_1)
print()


=== #1 ===
(1.2409373376568382, '!4.Requirements  \nThis document covers the following certification specifications requirements: \nTable 1\n Ð Requirements  \n5.Reference for centre of gravity values \nDatum plane:  leading edge at wing mean aerodynamic chord \nRequirement \nCS-LSA\n.15, 29th \nJuly 2013 \namendment 1 (ASTM F2245-12d)\nSubject of requirement\nReferenced \nchapter\n3.1Definitions:\nÐ3.1.2maximum empty weight\n, W\nE Ð largest empty weight of the \nairplane, including all operational equipment that is \ninstalled in the airplane: weight of the airframe, \npowerplant, required equipment, optional and specific \nequipment, fixed ballast, full engine coolant and oil, \nhydraulic fluid, and the unusable fuel.')



In [565]:
print("=== #2 ===")
print(show_at_2)
print()



=== #2 ===
(1.1018926050787448, 'Holding pattern entry procedures are not mandatory\n, they are merely a recommendation (as long as the airplane remains within the protected \nairspace).')



In [566]:
print("=== #3 ===")
print(show_at_3)



=== #3 ===
(1.2219378635300795, 'Instrument  Ground School 2015\nHolding PatternsDME leg length may be used \ninstead of time \nfor holding \npatterns.When used, DME leg length \nrefers to the \noutbound leg \nlength.')


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
articleURL = "http://curia.europa.eu/juris/document/document.jsf?text=&docid=139407&pageIndex=0&doclang=EN&mode=lst&dir=&occ=first&part=1&cid=52454"
def getText(url):
    page = urlopen(url).read().decode('utf8', 'ignore')
    soup = BeautifulSoup(page, 'lxml')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text.encode('ascii', errors='replace').decode().replace("?","")
text = getText(articleURL)

In [412]:
def process(text, n):
    text1 = summarize(text, n) 
    Z = Vect(text1, T)
    X = Z[0]
    km = Z[1]
    vectorizer = Z[2]
    new_post_vec = vectorizer.transform([post])
    similar = []
    for i in range(0, 100):
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, summaryArr[i]))
    similar = sorted(similar) 
    show_at_1 = similar[0]
    show_at_2 = similar[1]
    show_at_3 = similar[2]
    return [show_at_1, show_at_2, show_at_2]
        
    

In [ ]:
def final(data, post, n = 100):
    Complete = {}
    filename, type1 = os.path.splitext(data)
    if type1 == 'pdf':
        text = getPdf(data)
        Output = process(text)
        
        
    if type1 == ('ppt' or 'pptx'):
        text = #Complete for PPT
        Output = process(text)
    
    for i in range(1, 17):
        text = getPdf(str(i)+'.pdf')
        text1 = summarize(text, n) 
        T = Vect(text1)
        X = T[0]
        km = T[1]
        vectorizer = T[2]
        Complete[str(i)+'.pdf'] = X
        
        
        
        
        
        
    

In [335]:
text = getPdf(str(1)+'.pdf')

In [585]:
    Complete = {}
    for i in range(1, 16):
        print(str(i)+'.pdf')
        text1 = getPdf(str(i)+'.pdf')
        print(text1)
        #text = getPdf('')
        text2 = summarize(text1, 10) 
        Z = Vect(text2, 10)
        X = Z[0]
        km = Z[1]
        vectorizer = Z[2]
        Complete[str(i)+'.pdf'] = X

1.pdf
Instrument  Ground School 2015
Holding Patterns and ProceduresA holding pattern provides a protected ai
rspace for a safe operation during the hold. Pilots are expected to remain within the protected airspace (the holding side). One of the elements which caus
es unnecessary confusion and anxiety is 
the holding pattern entry. Holding pattern entry procedures are not mandatory
, they are merely a recommendation (as long as the airplane remains within the protected 
airspace). The recommendation is based on three types of entries, depending on the sector from which the airplane arri
ves at the holding fix. These are the direct, parallel and tear-dropentries.
Instrument  Ground School 2015

Instrument  Ground School 2015
Holding PatternsStandardholding 
pattern -all turns made 

to right.The inbound legof a holding pattern is 1 minuteat & below 14,000 ft. and 1.5 
minutes above 14,000 
ft.Outbound leg should be adjustedto yield a 1 
minute inbound leg.Timing of the outbound leg shou

1   Volume 16, Issue 19 November 14, 2008 Volume 16, Issue 19      
 
 
 
   
 
 
 
 
   
 
 
 
  A FOCUS ON SAFETY ALL:
 The Gulfstream manuals now call only for bonding 
the fuel source to the aircraft during fueling or defueling...2                                   TOP STORIES ALL: 
An overview of the large-cabin AMM revisions 
Technical Publications has produced to date in 2008–
3  
AstraŽ/SPXŽ/G100
®/G150®/G200®: Rudder Trim 
installations on the Astra through G200 aircraft are 
susceptible to water ingress into the trim actuator 
housings...4  G150®: ﬁGEN LOAD UNBALANCEﬂ status messages 
have been traced to a poor gr
ound for the APU generator 
return cable at 10GND...5  G200®: An issue surrounding the operational 
characteristics of the Entr
y Light Switch Panel (CMS1 
Cabin System) has been brought 
to our attention by a 
G200 operator...5  GII®/GIII®: During left engine start, when the start button 
was depressed, both engines began rotating...
6  GIV®/G300®/G400®: A GIV crew

In [586]:
# Updated Version is Below
"""
for l in range(2, 16):
    print(l)
    X = Complete[str(l)+'.pdf']

    similar = []
    for i in range(1, 10):
        print(i)
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, summaryArr[i]))
"""
    

2
1
2
3
4
5
6
7
8
9
3
1
2
3
4
5
6
7
8
9
4
1
2
3
4
5
6
7
8
9
5
1
2
3
4
5
6
7
8
9
6
1
2
3
4
5
6
7
8
9
7
1
2
3
4
5
6
7
8
9
8
1
2
3
4
5
6
7
8
9
9
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
11
1
2
3
4
5
6
7
8
9
12
1
2
3
4
5
6
7
8
9
13
1
2
3
4
5
6
7
8
9
14
1
2
3
4
5
6
7
8
9
15
1
2
3
4
5
6
7
8
9


In [597]:
for l in range(2, 16):
    name = str(l)+'.pdf'
    X = Complete[str(l)+'.pdf']

    similar = []
    for i in range(1, 10):
        dist = sp.linalg.norm((new_post_vec - X[i]).toarray())
        similar.append((dist, name, summaryArr[i]))

    

In [598]:
similar = sorted(similar) 
print(len(similar))
show_at_1 = similar[0]
show_at_2 = similar[1]
show_at_3 = similar[2]

9


In [599]:
print(show_at_1)

(1.2409373376568382, '15.pdf', '!4.Requirements  \nThis document covers the following certification specifications requirements: \nTable 1\n Ð Requirements  \n5.Reference for centre of gravity values \nDatum plane:  leading edge at wing mean aerodynamic chord \nRequirement \nCS-LSA\n.15, 29th \nJuly 2013 \namendment 1 (ASTM F2245-12d)\nSubject of requirement\nReferenced \nchapter\n3.1Definitions:\nÐ3.1.2maximum empty weight\n, W\nE Ð largest empty weight of the \nairplane, including all operational equipment that is \ninstalled in the airplane: weight of the airframe, \npowerplant, required equipment, optional and specific \nequipment, fixed ballast, full engine coolant and oil, \nhydraulic fluid, and the unusable fuel.')
